In [1]:
import cv2
import numpy as np
from sklearn.svm import LinearSVC
import pandas as pd
from tqdm import tqdm
from skimage.feature import hog

In [128]:
df_ground_truth = pd.read_csv('./train.csv')
df_ground_truth.head()

,frame_id,bounding_boxes
0,train/A_001.jpg,0 225 214 317 0 172 345 254 285 240 155 131 70...
1,train/A_002.jpg,0 254 190 293 0 169 338 271 276 238 160 137 70...
2,train/A_003.jpg,0 306 59 241 0 155 306 318 235 233 191 149 713...
3,train/A_004.jpg,0 143 239 298 164 223 240 172 721 293 94 76 57...
4,train/A_005.jpg,0 217 137 270 55 209 323 208 731 296 99 79 573...


In [130]:
# Check for nan in bounding boxes
df_ground_truth.isna().sum()

# Replace nan with [0.0,0.0,0.0,0.0]
df_ground_truth = df_ground_truth.fillna('0 0 0 0')

In [30]:
# Step 1: Extract HOG features from the training images
def extract_features(img):
    features, hog_image = hog(img, orientations=12, pixels_per_cell=(8, 8),cells_per_block=(3, 3), visualize=True, channel_axis=-1)
    return features

# Step 2: Label the HOG features
def label_features(features, bounding_boxes):
    labels = []
    for box in bounding_boxes:
        labels.append(1)
    return features, labels

# Step 3: Train a classifier
def train_classifier(features, labels):
    clf = LinearSVC()
    clf.fit(features, labels)
    return clf

# Step 4: Extract HOG features from new images
def extract_features_from_image(img):
    features = extract_features(img)
    return features

# Step 5: Use the classifier to detect vehicles in new images
def detect_vehicles(img, clf):
    features = extract_features_from_image(img)
    predictions = clf.predict(features)
    return predictions

# Step 6: Draw the bounding boxes around the detected vehicles
def draw_bounding_boxes(img, predictions, bounding_boxes):
    for i, pred in enumerate(predictions):
        if pred == 1:
            x, y, w, h = bounding_boxes[i]
            cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)
    return img

def annotations_for_frame(df_annotation, frame):
    assert frame in df_annotation.index
    bbs = df_annotation[df_annotation.index == frame].bounding_boxes.values[0]

    if pd.isna(bbs): # some frames contain no vehicles
        return []

    bbs = list(map(lambda x : int(x), bbs.split(' ')))
    return np.array_split(bbs, len(bbs) / 4)

In [131]:
# Load the training images and their bounding boxes
training_images = []
bounding_boxes = []

for i in tqdm(df_ground_truth.index):
    # Save frame_id
    frame_id = df_ground_truth.loc[i, 'frame_id']

    # Save image
    img = cv2.imread(frame_id)

    # Save bounding box
    train_labels = annotations_for_frame(df_ground_truth, i)

    training_images.append(img)
    bounding_boxes.append(train_labels)

100%|██████████| 2222/2222 [00:36<00:00, 61.37it/s]


In [ ]:
# Extract HOG features and labels
hog_features, labels = tqdm(label_features([extract_features(img) for img in training_images], [box for box in bounding_boxes]))

In [132]:
# Concat hog_features as new column for df_ground_truth
df_ground_truth['hog_features'] = hog_features

In [133]:
# Transform string for each bounding_boxes to a list of floats
df_ground_truth['bounding_boxes'] = df_ground_truth['bounding_boxes'].apply(lambda x: np.array([float(i) for i in str(x).split(' ')]))
df_ground_truth.head()

,frame_id,bounding_boxes,hog_features
0,train/A_001.jpg,"[0.0, 225.0, 214.0, 317.0, 0.0, 172.0, 345.0, ...","[0.2371046026703983, 0.2371046026703983, 0.110..."
1,train/A_002.jpg,"[0.0, 254.0, 190.0, 293.0, 0.0, 169.0, 338.0, ...","[0.05479810084769836, 0.029798387772276894, 0...."
2,train/A_003.jpg,"[0.0, 306.0, 59.0, 241.0, 0.0, 155.0, 306.0, 3...","[0.06389607869122489, 0.019384123841706187, 0...."
3,train/A_004.jpg,"[0.0, 143.0, 239.0, 298.0, 164.0, 223.0, 240.0...","[0.022722701559013465, 0.0026056156956097525, ..."
4,train/A_005.jpg,"[0.0, 217.0, 137.0, 270.0, 55.0, 209.0, 323.0,...","[0.047339984197445666, 0.10008659581998491, 0...."


In [134]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# Split the data into training/testing sets
X_train, X_test, y_train, y_test = train_test_split(df_ground_truth['hog_features'], df_ground_truth['bounding_boxes'], test_size=0.3, random_state=42)

In [174]:
test_features = np.array([[2.0,3.0,9.0,8.4],[2.0,3.0,9.0,7.4]])
test_target = np.array([[3.0,5.3],[4.0,0.0]])

# Create linear regression object
regr = LinearRegression()

# Train the model using the training sets
regr.fit(test_features, test_target)

LinearRegression()

In [165]:
y_train = np.asarray(y_train)
y_train.shape

NameError: name 'narray' is not defined

In [164]:
# Multivariate Linear Regression
from sklearn.linear_model import LinearRegression

# Create linear regression object
regr = LinearRegression()

# Train the model using the training sets
regr.fit(X_train, y_train)

# Make predictions using the testing set
y_pred = regr.predict(X_test)

# The coefficients
print('Coefficients: \n', regr.coef_)
# The mean squared error
print('Mean squared error: %.2f'
        % mean_squared_error(y_test, y_pred))
# The coefficient of determination: 1 is perfect prediction
print('Coefficient of determination: %.2f'
        % r2_score(y_test, y_pred))

# Plot outputs
plt.scatter(X_test, y_test,  color='black')
plt.plot(X_test, y_pred, color='blue', linewidth=3)

plt.xticks(())
plt.yticks(())
plt.show()

ValueError: setting an array element with a sequence.

In [ ]:
# Train the classifier
clf = tqdm(train_classifier(hog_features, labels))

# Read a new image
img = cv2.imread("./test/005.jpg")

# Detect vehicles in the new image
predictions = detect_vehicles(img, clf)

# Draw bounding boxes around the detected vehicles
final_image = draw_bounding_boxes(img, predictions, bounding_boxes)

# Show the final image
cv2.imshow("Image",img)
cv2.imshow("Detected Vehicles", final_image)
cv2.waitKey(0)
cv2.destroyAllWindows()